<a href="https://colab.research.google.com/github/yaswhar/learning-genesis-sim/blob/main/Control%20Your%20Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install genesis-world -q
!pip install mediapy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
ICD_CONFIG_CONTENT = """{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
"""
with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write(ICD_CONFIG_CONTENT)

In [3]:
import genesis as gs
import numpy as np
import mediapy as media
from tqdm.notebook import tqdm
## Initialization
gs.init(backend=gs.cuda)

[Genesis] [14:36:28] [INFO] ╭───────────────────────────────────────────────╮
INFO:genesis:~<╭───────────────────────────────────────────────╮>~
[Genesis] [14:36:28] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
INFO:genesis:~<│┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈>~ ~~~~<Genesis>~~~~ ~<┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│>~
[Genesis] [14:36:28] [INFO] ╰───────────────────────────────────────────────╯
INFO:genesis:~<╰───────────────────────────────────────────────╯>~
[Genesis] [14:36:28] [INFO] Running on [Tesla T4] with backend gs.cuda. Device memory: 14.74 GB.
INFO:genesis:Running on ~~<[Tesla T4]>~~ with backend ~~<gs.cuda>~~. Device memory: ~~<14.74>~~ GB.
[Genesis] [14:36:29] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
INFO:genesis:🚀 Genesis initialized. 🔖 version: ~~<0.2.1>~~, 🌱 seed: ~~<None>~~, 📏 precision: '~~<32>~~', 🐛 debug: ~~<False>~~, 🎨 theme: '~~<dark>~~'.


In [4]:
## Create a Scene
scene = gs.Scene(
    viewer_options = gs.options.ViewerOptions(
        camera_pos    = (0, -3.5, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 30,
        res           = (960, 640),
        max_FPS       = 60,
    ),
    sim_options = gs.options.SimOptions(
        dt = 0.01,
    ),
    show_viewer = False,
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)

[Genesis] [14:36:33] [INFO] Scene <d386b89> created.
INFO:genesis:Scene ~~~<<d386b89>>~~~ created.


In [5]:
## Entities
plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(
        file  = 'xml/franka_emika_panda/panda.xml',
    ),
)

[Genesis] [14:36:46] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <2ad187d>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
INFO:genesis:Adding ~<<gs.RigidEntity>>~. idx: ~<0>~, uid: ~~~<<2ad187d>>~~~, morph: ~<<gs.morphs.Plane>>~, material: ~<<gs.materials.Rigid>>~.
[Genesis] [14:36:46] [INFO] Preprocessing geom idx 0.
INFO:genesis:Preprocessing geom idx ~~<0>~~.
[Genesis] [14:36:46] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <4455994>, morph: <gs.morphs.MJCF(file='/usr/local/lib/python3.11/dist-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
INFO:genesis:Adding ~<<gs.RigidEntity>>~. idx: ~<1>~, uid: ~~~<<4455994>>~~~, morph: ~<<gs.morphs.MJCF(file='/usr/local/lib/python3.11/dist-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>>~, material: ~<<gs.materials.Rigid>>~.
[Genesis] [14:36:47] [INFO] Preprocessing geom idx 1.
INFO:genesis:Preprocessing geom idx ~~<1>~~.
[Genesis] [14:36:54] [INFO] Preprocessing geom idx 2.


Note that here we are using `.dof_idx_local` to obtain the local idx of the dof with respect to the robot entity itself. You can also use `joint.dof_idx` to access each joint’s global dof index in the scene.

In [6]:
## Build
scene.build()

jnt_names = [
    'joint1',
    'joint2',
    'joint3',
    'joint4',
    'joint5',
    'joint6',
    'joint7',
    'finger_joint1',
    'finger_joint2',
]
dofs_idx = [franka.get_joint(name).dof_idx_local for name in jnt_names]

[Genesis] [14:37:20] [INFO] Building scene <d386b89>...
INFO:genesis:Building scene ~~~<<d386b89>>~~~...
[Genesis] [14:37:35] [INFO] Compiling simulation kernels...
INFO:genesis:Compiling simulation kernels...
[Genesis] [14:38:32] [INFO] Building visualizer...
INFO:genesis:Building visualizer...


Usually, gains are set in the URDF/MJCF file but they can be tuned manually or referred to a well-tuned value online.

In [7]:
## Set Control Gains
# set positional gains
franka.set_dofs_kp(
    kp             = np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
    dofs_idx_local = dofs_idx,
)
# set velocity gains
franka.set_dofs_kv(
    kv             = np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
    dofs_idx_local = dofs_idx,
)
# set force range for safety
franka.set_dofs_force_range(
    lower          = np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    upper          = np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
    dofs_idx_local = dofs_idx,
)

In [10]:
# Hard Reset
cam.start_recording()

frames_reset = []
scene.reset()
for i in range(150):
    if i < 50:
        franka.set_dofs_position(np.array([1, 1, 0, 0, 0, 0, 0, 0.04, 0.04]), dofs_idx)
    elif i < 100:
        franka.set_dofs_position(np.array([-1, 0.8, 1, -2, 1, 0.5, -0.5, 0.04, 0.04]), dofs_idx)
    else:
        franka.set_dofs_position(np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]), dofs_idx)

    scene.step()
    frames_reset.append(cam.render()[0])

[Genesis] [14:40:04] [INFO] Resetting Scene <d386b89>.
INFO:genesis:Resetting Scene ~~~<<d386b89>>~~~.
[Genesis] [14:40:04] [INFO] Running at 0.60 FPS.
INFO:genesis:Running at ~<0.60>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.63 FPS.
INFO:genesis:Running at ~<0.63>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.67 FPS.
INFO:genesis:Running at ~<0.67>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.70 FPS.
INFO:genesis:Running at ~<0.70>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.74 FPS.
INFO:genesis:Running at ~<0.74>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.77 FPS.
INFO:genesis:Running at ~<0.77>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.81 FPS.
INFO:genesis:Running at ~<0.81>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.85 FPS.
INFO:genesis:Running at ~<0.85>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.90 FPS.
INFO:genesis:Running at ~<0.90>~ FPS.
[Genesis] [14:40:04] [INFO] Running at 0.94 FPS.
INFO:genesis:Running at ~<0.94>~ FPS.
[Genesis] [14:40:04] [INFO]

In [12]:
# Show Hard Reset
media.show_video(frames_reset, fps=50)

This browser does not support the video tag.


In [13]:
# PD control
frames = []

for i in range(1250):
    if i == 0:
        franka.control_dofs_position(
            np.array([1, 1, 0, 0, 0, 0, 0, 0.04, 0.04]),
            dofs_idx,
        )
    elif i == 250:
        franka.control_dofs_position(
            np.array([-1, 0.8, 1, -2, 1, 0.5, -0.5, 0.04, 0.04]),
            dofs_idx,
        )
    elif i == 500:
        franka.control_dofs_position(
            np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
            dofs_idx,
        )
    elif i == 750:
        # control first dof with velocity, and the rest with position
        franka.control_dofs_position(
            np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])[1:],
            dofs_idx[1:],
        )
        franka.control_dofs_velocity(
            np.array([1.0, 0, 0, 0, 0, 0, 0, 0, 0])[:1],
            dofs_idx[:1],
        )
    elif i == 1000:
        franka.control_dofs_force(
            np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
            dofs_idx,
        )
    # This is the control force computed based on the given control command
    # If using force control, it's the same as the given control command
    print('control force:', franka.get_dofs_control_force(dofs_idx))

    # This is the actual force experienced by the dof
    print('internal force:', franka.get_dofs_force(dofs_idx))

    scene.step()
    frames.append(cam.render()[0])

control force: 

[Genesis] [14:44:30] [INFO] Running at 0.08 FPS.
INFO:genesis:Running at ~<0.08>~ FPS.
[Genesis] [14:44:30] [INFO] Running at 0.08 FPS.
INFO:genesis:Running at ~<0.08>~ FPS.
[Genesis] [14:44:30] [INFO] Running at 0.08 FPS.
INFO:genesis:Running at ~<0.08>~ FPS.
[Genesis] [14:44:30] [INFO] Running at 0.09 FPS.
INFO:genesis:Running at ~<0.09>~ FPS.
[Genesis] [14:44:30] [INFO] Running at 0.09 FPS.
INFO:genesis:Running at ~<0.09>~ FPS.
[Genesis] [14:44:30] [INFO] Running at 0.10 FPS.
INFO:genesis:Running at ~<0.10>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.10 FPS.
INFO:genesis:Running at ~<0.10>~ FPS.


tensor([ 87.0000,  87.0000,   2.4990,  87.0000, -12.0000,  12.0000,   1.3465,
          3.9794,   4.0206], device='cuda:0')
internal force: tensor([ 0.0000e+00,  4.0399e+00,  1.0302e-07,  3.2669e+00, -1.4812e-09,
        -2.2997e+00, -6.4067e-10, -2.6404e-09,  2.6404e-09], device='cuda:0')
control force: tensor([ 8.7000e+01,  8.7000e+01,  1.1395e+00,  8.7000e+01, -7.3143e+00,
         1.2000e+01, -5.2077e-02,  2.0460e+00,  2.0428e+00], device='cuda:0')
internal force: tensor([ 87.0123,  91.5943,   2.5109,  91.6631, -12.0872,  10.0559,   1.3528,
          3.9796,   4.0204], device='cuda:0')
control force: tensor([87.0000, 87.0000,  1.1691, 87.0000,  0.3706, 12.0000, -0.1400,  0.9669,
         0.9412], device='cuda:0')
internal force: tensor([ 8.6822e+01,  9.1422e+01,  1.1480e+00,  9.1288e+01, -7.3614e+00,
         9.9989e+00, -5.3183e-02,  1.8717e+00,  1.8616e+00], device='cuda:0')
control force: tensor([87.0000, 87.0000,  1.3523, 87.0000,  0.9198, 12.0000, -0.1671,  0.3713,
         0.

[Genesis] [14:44:31] [INFO] Running at 0.11 FPS.
INFO:genesis:Running at ~<0.11>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.11 FPS.
INFO:genesis:Running at ~<0.11>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.12 FPS.
INFO:genesis:Running at ~<0.12>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.13 FPS.
INFO:genesis:Running at ~<0.13>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.13 FPS.
INFO:genesis:Running at ~<0.13>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.14 FPS.
INFO:genesis:Running at ~<0.14>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.15 FPS.
INFO:genesis:Running at ~<0.15>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.16 FPS.
INFO:genesis:Running at ~<0.16>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.16 FPS.
INFO:genesis:Running at ~<0.16>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.17 FPS.
INFO:genesis:Running at ~<0.17>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.18 FPS.
INFO:genesis:Running at ~<0.18>~ FPS.
[Genesis] [14:44:31] [INFO] Running at 0.19

control force: tensor([87.0000, 87.0000,  1.5310, 87.0000,  0.8589,  1.9254, -0.1514, -0.2278,
        -0.2604], device='cuda:0')
internal force: tensor([ 8.5694e+01,  9.1540e+01,  1.3297e+00,  8.9337e+01,  8.4048e-01,
         1.7084e-02, -1.6267e-01, -4.4790e-01, -5.0614e-01], device='cuda:0')
control force: tensor([87.0000, 87.0000,  1.5154, 87.0000,  0.7206,  2.1675, -0.1441, -0.2278,
        -0.2706], device='cuda:0')
internal force: tensor([85.4089, 91.7878,  1.1921, 89.0033,  0.6568, -0.2362, -0.1522, -0.4540,
        -0.5192], device='cuda:0')
control force: tensor([87.0000, 87.0000,  1.6037, 87.0000,  0.6864,  2.3302, -0.1392, -0.2150,
        -0.2666], device='cuda:0')
internal force: tensor([ 8.5075e+01,  9.2129e+01,  1.0316e+00,  8.8735e+01,  4.4761e-01,
         4.8082e-02, -1.4457e-01, -4.2790e-01, -5.1183e-01], device='cuda:0')
control force: tensor([87.0000, 87.0000,  1.7743, 87.0000,  0.7295,  2.4104, -0.1360, -0.1985,
        -0.2535], device='cuda:0')
internal force:

...

control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.4286e-04,  4.8869e+01, -7.2543e-01, -1.8053e+01, -2.0241e-01,
         2.3206e+00, -8.5358e-05,  2.1784e-02, -2.1784e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.3937e-04,  4.8869e+01, -7.2652e-01, -1.8053e+01, -2.0173e-01,
         2.3200e+00, -8.6941e-05,  2.1784e-02, -2.1784e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.3505e-04,  4.8870e+01, -7.2760e-01, -1.8053e+01, -2.0104e-01,
         2.3195e+00, -8.8498e-05,  2.1784e-02, -2.1784e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.3421e-04,  4.8870e+01, -7.2868e-01, -1.8053e+01, -2.0036e-01,
         2.3189e+00, -8.9444e-05,  2.1784e-02, -2.1783e-02], device='cuda:0')
control force: tensor([0

[Genesis] [14:45:04] [INFO] Running at 34.32 FPS.
INFO:genesis:Running at ~<34.32>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 33.95 FPS.
INFO:genesis:Running at ~<33.95>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 33.95 FPS.
INFO:genesis:Running at ~<33.95>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 34.17 FPS.
INFO:genesis:Running at ~<34.17>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 34.40 FPS.
INFO:genesis:Running at ~<34.40>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 34.64 FPS.
INFO:genesis:Running at ~<34.64>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 34.96 FPS.
INFO:genesis:Running at ~<34.96>~ FPS.


control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.4621e-04,  4.8871e+01, -7.3077e-01, -1.8053e+01, -1.9896e-01,
         2.3179e+00, -8.5667e-05,  2.1784e-02, -2.1783e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.5424e-04,  4.8871e+01, -7.3180e-01, -1.8053e+01, -1.9825e-01,
         2.3173e+00, -8.2701e-05,  2.1784e-02, -2.1784e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.6257e-04,  4.8871e+01, -7.3282e-01, -1.8054e+01, -1.9754e-01,
         2.3168e+00, -7.9792e-05,  2.1784e-02, -2.1783e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.6363e-04,  4.8872e+01, -7.3385e-01, -1.8054e+01, -1.9683e-01,
         2.3163e+00, -7.8954e-05,  2.1784e-02, -2.1784e-02], device='cuda:0')
control force: tensor([0

[Genesis] [14:45:04] [INFO] Running at 35.36 FPS.
INFO:genesis:Running at ~<35.36>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 35.02 FPS.
INFO:genesis:Running at ~<35.02>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 34.83 FPS.
INFO:genesis:Running at ~<34.83>~ FPS.
[Genesis] [14:45:04] [INFO] Running at 35.15 FPS.
INFO:genesis:Running at ~<35.15>~ FPS.
[Genesis] [14:45:05] [INFO] Running at 35.40 FPS.
INFO:genesis:Running at ~<35.40>~ FPS.


control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.4663e-04,  4.8873e+01, -7.3804e-01, -1.8054e+01, -1.9403e-01,
         2.3141e+00, -7.8633e-05,  2.1785e-02, -2.1785e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.4273e-04,  4.8874e+01, -7.3908e-01, -1.8054e+01, -1.9333e-01,
         2.3136e+00, -7.8257e-05,  2.1785e-02, -2.1784e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.4060e-04,  4.8874e+01, -7.4010e-01, -1.8054e+01, -1.9262e-01,
         2.3130e+00, -7.7579e-05,  2.1786e-02, -2.1785e-02], device='cuda:0')
control force: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
internal force: tensor([-1.3921e-04,  4.8874e+01, -7.4113e-01, -1.8054e+01, -1.9191e-01,
         2.3125e+00, -7.6286e-05,  2.1786e-02, -2.1786e-02], device='cuda:0')


In [14]:
# Show Frames
media.show_video(frames, fps=60)

This browser does not support the video tag.
